In [14]:
from pymystem3 import Mystem # for normalization of text
my_stem = Mystem()

import time

from ipywidgets import IntProgress
from IPython.display import display


import pandas as pd
from collections import defaultdict

import gensim.models

DATA_PATH = '/data/'

CAPTION_PATH = DATA_PATH + 'captions/'
EVENTS_PATH = DATA_PATH + 'events/'


cities = ['spb', 'moscow']
years = ['2019', '2020']
posts_types = ['raw/', 'cleaned/', 'lem/']

def events_path(path, city):
    return path + city + '_events.csv'

def posts_path(path, city, year):
    return path + city + '_posts_' + year + '.csv'

In [38]:
CAPTION_PATH = DATA_PATH + 'captions/'

preprocessing for events

In [31]:
def read_events(path: str, city: str):
    df = pd.read_csv(events_path(path, city))
    df['city'] = [city] * len(df)
    return df

df_events = pd.concat([read_events(EVENTS_PATH, city) for city in cities])

In [25]:
events_codes = [ids[2:-2].split("', '") for ids in df_events['postcodes']]
codes_to_ids = [[(code, event_id) for code in codes] for event_id, codes in enumerate(events_codes)]
code_to_event = dict(sum(codes_to_ids, []))

In [40]:
SPLITTER = ' &+& '

def process_posts_csv(path: str, captions: list):
    df = pd.read_csv(path)
    df = df[df['code'].isin(code_to_event)]
    for ind, row in df.iterrows():
        captions[code_to_event[row['code']]].append(row['caption'])
    return captions

def process_dir(posts_type: str, cities: list, years: list):
    captions = [[] for i in range(len(df_events))]
    for city in cities:
        for year in years:
            path_csv = posts_path(f'{CAPTION_PATH}{posts_type}', city, year)
            captions = process_posts_csv(path_csv, captions)
    
    df_events['caption'] = captions
    df_events['description'] = [SPLITTER.join(posts) for posts in captions]
    df_events.to_csv(r'' + EVENTS_PATH + posts_type + 'events.csv', index=False)
            
    

for pt in posts_types:
    process_dir(pt, cities, years)